In [109]:
from dataclasses import dataclass, field, asdict

@dataclass
class OutputContainer:
    generated_data: list = field(init=False)
    real_data: list = field(init=False)
    y_real: list = field(init=False)
    y_fake: list = field(init=False)
    loss_generator: list = field(init=False)
    loss_critic: list = field(init=False)
    loss_sp_critic: list = field(init=False)
    gradient_penalty: list = field(init=False)

    def clear(self):
        for key, value in asdict(self).items():
            print(key, value)
            if isinstance(value, list):
                setattr(self, key, [])
                

In [110]:
container = OutputContainer()

In [112]:
container.generated_data = [1, 2, 3]
container.real_data = [4, 5, 6]
container.y_real = [7, 8, 9]
container.y_fake = [10, 11, 12]
container.loss_generator = [13, 14, 15]
container.loss_critic = [16, 17, 18]
container.loss_sp_critic = [19, 20, 21]
container.gradient_penalty = [22, 23, 24]


In [97]:
dir(container)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'clear',
 'data',
 'loss_generator']

In [113]:
container.clear()

generated_data [1, 2, 3]
real_data [4, 5, 6]
y_real [7, 8, 9]
y_fake [10, 11, 12]
loss_generator [13, 14, 15]
loss_critic [16, 17, 18]
loss_sp_critic [19, 20, 21]
gradient_penalty [22, 23, 24]


In [100]:
container.data = [1, 2, 3, 4 ,5]

In [114]:
container.clear()

generated_data []
real_data []
y_real []
y_fake []
loss_generator []
loss_critic []
loss_sp_critic []
gradient_penalty []


In [67]:
print(container)

OutputContainer()


In [105]:
cont = dict()

In [106]:
cont['data'] = [1, 2, 3, 4, 5]

In [108]:
cont['data']

[1, 2, 3, 4, 5]